
L'obiettivo del progetto è apprendere mediante una rete neurale la trasformazione da punti del piano espressi in coordinate polari ad una rappresentazione basata su di una griglia discreta di dimensione 10x10, dove la cella della griglia ha valore 1 se contiene il punto, e 0 altrimenti.

Il dataset supervisionato è fornito in questo notebook nella forma di una generatore. Il generatore deve essere considerato come una "scatola nera" il cui comportamento deve essere appreso. 

Dovete progettare una rete neurale in grado di raggiungere una accuratezza del 95%. Questa è una condizione necessaria per superare l'esame, ma l'accuratezza non influisce in altro modo sulla valutazione.  

I modelli che raggiungono l'accuratezza attesa saranno invece valutati in modo inversamente proporzionale al numero dei loro parametri: **più il modello è piccolo, meglio è.**


**Attenzione**: Qualunque soluzione che tragga vantaggio, diretto o indiretto, da meta-conoscenza relativa al generatore sarà automaticamente bocciato.


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model

2023-01-10 15:04:18.176927: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Veniamo al generatore. Questo restituisce delle triple della forma
((theta,rho),out) dove (theta,rho) sono le coordinate polari di un punto nel primo quadrante del piano, e out è una mappa 10x10 con "1" in correspondenza alla cella che contiene il punto, e "0" altrimenti.

Settando  flat=True, la mappa 10x10 viene appiattita ad un vettore di dimensione 100. Potete utilizzare questa variante, se preferite. Nessuna altra modifica del generatore è ammessa. 

In [2]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

Creiamo una istanza del generatore con una griglia di dimensione 3x4

In [3]:
g1,g2 = 3,4
gen = polar_generator(1,grid=(g1,g2),noise=0.0)

... e osserviamo qualche esempio

In [5]:
# (theta,rho),maps = next(gen)
# for i,map in enumerate(maps):
#   #let us compute the cartesian coordinates
#   x = np.cos(theta[i])*rho[i]
#   y = np.sin(theta[i])*rho[i]
#   print("x coordinate (row): {}".format(int(x*g1)))
#   print("y coordinate (col): {}".format(int(y*g2)))
#   print("map:")
#   print(np.reshape(map,(g1,g2)))

x coordinate (row): 1
y coordinate (col): 3
map:
[[0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]


Utile esercizio: aggiungete rumore al generatore e verificate l'effetto sulla
"ground truth".

# Cosa consegnare

Ai fini del progetto dovete lavorare con la **griglia di default di dimensione 10x10, e con il rumore di default .002**

il generatore deve essere trattato come una scatola nera: non modificatelo e non sfruttate la sua semantica, che si suppone ignota. Potete lavorare in modlaità "flat", se preferite.

Dovete:

1.   definire una funzione per il calcolo della accuratezza (potete prendere ispirazione dal cocice della cella precedente) 
2.   definire una rete neurale che prende in input theta e rho e restituisce out
3.  misurare l'accuratezza della rete, che deve essere maggiore o uguale del 95%; l'accuratezza deve essere misurata su almeno 20000 dati
4. perfezionare il modello cercando di diminuire il più possibile il numero dei parametri mantenendo una accuratezza superiore al 95%. Solo la vostra rete migliore deve essere consegnata.

Dovete consegnare un UNICO notebook eseguibile su colab, che contenga il codice della rete, il suo sommario con il numero dei parametri, la storia di training, il codice per il calcolo della accuratezza e la sua valutazione sulla vostra rete.

**N.B.** L'accuratezza deve essere superiore o uguale a 95%, ma non influisce in altro modo sulla valutazione. Il vostro punteggio dipenderà unicamente dal numero dei parametri: più è piccolo e più la vostra vaalutazione sarà elevata.  

#Buon lavoro!





In [128]:
import keras
import numpy as np
from keras import layers
from keras import activations
from keras import losses
from sklearn.model_selection import train_test_split
import keras.backend as K

size = 10

In [129]:
# def map_map(map: np.ndarray) -> np.ndarray:
#   out = np.zeros(size * 2)
#   point = np.argwhere(map==1)[0]
#   out[point[0]] = 1
#   out[size + point[1]] = 1
#   return out

def map_map(map: np.ndarray) -> np.ndarray:
  out = np.zeros(size * 2)
  point = np.argwhere(map==1)[0]
  return point

In [235]:
batch_size = 512

In [236]:
n_train = 100000
n_test = 20000

def xy(i):
  (theta,rho),y = i
  res = map(map_map, y)
  y = np.array(list(res))
  x=np.array([i for i in zip(theta,rho)])
  return (x,y)

gcd = np.gcd(n_train, n_test)
gen = polar_generator(gcd,grid=(size,size),noise=0.02)

x_train, y_train = xy(next(gen))
for i in range(int(n_train/gcd)-1):
  x,y = xy(next(gen))
  x_trian = np.concatenate((x, x_train), axis=0)
  y_trian = np.concatenate((y, y_train), axis=0)

x_test, y_test = xy(next(gen))
for i in range(int(n_train/gcd)-1):
  x,y = xy(next(gen))
  x_trian = np.concatenate((x, x_test), axis=0)
  y_trian = np.concatenate((y, y_test), axis=0)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=n_test/(n_train+n_test), shuffle=True, random_state=1)

In [237]:
def loss_fn(y_true, y_pred):
  point = np.array(y_true).argwhere(x==1)[1]

  x = K.mult(K.cos(y_true[:1]), y_true[:2])
  #x = np.cos(theta[i])*rho[i]
  #y = np.sin(theta[i])*rho[1,164  
def point_accuracy(y_true: tf.Tensor, y_pred: tf.Tensor) -> float:
  norm = tf.cast(tf.round(y_pred), tf.uint8)
  return tf.equal(tf.cast(y_true, tf.uint8), norm)

In [241]:
model = keras.Sequential([
    layers.Flatten(),
    layers.Dense(100, activation=activations.swish),
    layers.Dense(50, activation=activations.sigmoid),
    layers.Dense(25, activation=activations.swish),
    layers.Dense(50, activation=activations.swish),
    layers.Dense(75, activation=activations.swish),
    layers.Dense(100, activation=activations.relu),

    layers.Dense(2, activation=activations.relu)
])
model.build((None, 2))
model.summary()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=losses.MeanAbsoluteError(),
    metrics=['accuracy', point_accuracy]
)

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_47 (Flatten)        (None, 2)                 0         
                                                                 
 dense_209 (Dense)           (None, 100)               300       
                                                                 
 dense_210 (Dense)           (None, 50)                5050      
                                                                 
 dense_211 (Dense)           (None, 25)                1275      
                                                                 
 dense_212 (Dense)           (None, 50)                1300      
                                                                 
 dense_213 (Dense)           (None, 75)                3825      
                                                                 
 dense_214 (Dense)           (None, 100)             

In [242]:
history = model.fit(
    x_train,
    y_train,
    epochs=100,# 300,
    batch_size=batch_size
)

Epoch 1/100
40/40 [==============================] - 0s 3ms/step - loss: 3.2208 - accuracy: 0.5361 - point_accuracy: 0.1016
Epoch 2/100
40/40 [==============================] - 0s 3ms/step - loss: 2.3507 - accuracy: 0.4927 - point_accuracy: 0.1041
Epoch 3/100
40/40 [==============================] - 0s 3ms/step - loss: 1.8809 - accuracy: 0.5396 - point_accuracy: 0.1519
Epoch 4/100
40/40 [==============================] - 0s 3ms/step - loss: 1.0108 - accuracy: 0.8780 - point_accuracy: 0.3885
Epoch 5/100
40/40 [==============================] - 0s 3ms/step - loss: 0.3838 - accuracy: 0.9434 - point_accuracy: 0.7130
Epoch 6/100
40/40 [==============================] - 0s 4ms/step - loss: 0.2948 - accuracy: 0.9528 - point_accuracy: 0.8284
Epoch 7/100
40/40 [==============================] - 0s 4ms/step - loss: 0.2674 - accuracy: 0.9531 - point_accuracy: 0.8710
Epoch 8/100
40/40 [==============================] - 0s 3ms/step - loss: 0.2573 - accuracy: 0.9510 - point_accuracy: 0.8865
Epoch 9/

In [243]:
score, _, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print(x_test.shape, y_test.shape)
print('Test score:', score)
print('Accuracy: {:.1f}%'.format(acc*100))

accs = []
for x in range(50):
  (theta,rho),y = next(gen)
  x=np.array([i for i in zip(theta,rho)])
  y = np.array(list(map(map_map, y)))

  score, _, acc = model.evaluate(x, y, batch_size=batch_size)
  accs.append(acc)

acc = np.mean(accs)
print('Accuracy: {:.1f}%'.format(acc*100))

40/40 [==============================] - 0s 2ms/step - loss: 0.2362 - accuracy: 0.9625 - point_accuracy: 0.9185
(20000, 2) (20000, 2)
Test score: 0.2361559122800827
Accuracy: 91.9%
40/40 [==============================] - 0s 3ms/step - loss: 0.2616 - accuracy: 0.9320 - point_accuracy: 0.8393
Accuracy: 82.9%


In [223]:
x, y = xy(next(gen))
for i in range(100):
  y_pred = model.predict(x[i:i+1], verbose=0)
  ans = np.round(y_pred)[0]
  if(~np.equal(y[i], ans).all()):
    print(y[i], y_pred[0], ans)

[1 8] [1.6918747 7.9072466] [2. 8.]
[7 1] [7.72345   0.5811634] [8. 1.]
[5 6] [5.5203557 5.820051 ] [6. 6.]
[2 3] [2.7413785 2.9114342] [3. 3.]
[5 6] [5.615502 6.330038] [6. 6.]
[4 2] [4.550197  2.4694378] [5. 2.]
[4 8] [4.6336503 8.546014 ] [5. 9.]
[2 8] [1.866356 8.577712] [2. 9.]
[6 2] [6.5518494 2.28608  ] [7. 2.]
[8 7] [8.512311 7.015404] [9. 7.]
[2 3] [2.781919 2.759175] [3. 3.]
[7 3] [7.6382375 2.8167334] [8. 3.]
[6 9] [6.6883197 9.252294 ] [7. 9.]
[9 3] [9.509774  3.1504257] [10.  3.]
[5 1] [5.6179605 1.3603102] [6. 1.]
[8 1] [8.25647    0.46758547] [8. 0.]
[1 2] [0.86550725 2.729677  ] [1. 3.]
[3 7] [3.0359097 7.5240703] [3. 8.]
[3 0] [3.5369406 0.2683025] [4. 0.]
[4 5] [4.6773305 5.1505375] [5. 5.]
[6 5] [6.8044896 5.196588 ] [7. 5.]
[0 0] [0.         0.76221025] [0. 1.]
[4 0] [4.641539   0.46320155] [5. 0.]
[3 1] [3.7650893 1.3877875] [4. 1.]
[7 2] [7.8303537 2.2623384] [8. 2.]
[7 9] [7.863147 9.293246] [8. 9.]
[0 6] [0.       6.778114] [0. 7.]
[9 5] [9.587254 4.662373] [10.